In [2]:
import pandas as pd

In [4]:
train_path = "/Users/kahaan/Desktop/multi-armed-bandits/netflix/TrainingRatings.txt"
test_path = "/Users/kahaan/Desktop/multi-armed-bandits/netflix/TestingRatings.txt"
labels_path = "/Users/kahaan/Desktop/multi-armed-bandits/netflix/netflix_genres.csv"
title_path = "/Users/kahaan/Desktop/multi-armed-bandits/netflix/movie_titles.txt"

titles = []
with open(title_path, 'r', encoding='latin1') as file:
    for line in file:
        parts = line.strip().split(',', 2)
        if len(parts) == 3:
            titles.append(parts)
        else:
            print(f"Skipped malformed line: {line}")

df_titles = pd.DataFrame(titles, columns=['MovieID', 'Year', 'Title'])
df_train = pd.read_csv(train_path, header=None, names=["MovieID", "UserID", "Rating"])
df_test = pd.read_csv(test_path, header=None, names=["MovieID", "UserID", "Rating"])
df_combined = pd.concat([df_train, df_test], ignore_index=True)
df_labels = pd.read_csv(labels_path, names=["MovieID", "Genres"]).drop(index=0)

df_combined['MovieID'] = df_combined['MovieID'].astype(int)
df_labels['MovieID'] = df_labels['MovieID'].astype(int)
df_titles['MovieID'] = df_titles['MovieID'].astype(int)

df_merge1 = pd.merge(df_combined, df_labels, on='MovieID', how='inner')
df_merge2 = pd.merge(df_merge1, df_titles, on='MovieID', how='inner')

entries_per_user = df_merge2.groupby('UserID').size()
smallest_entries_count = entries_per_user.min()
print(f"The smallest amount of entries for any UserID is: {smallest_entries_count}")

df_merge2

The smallest amount of entries for any UserID is: 39


,MovieID,UserID,Rating,Genres,Year,Title
0,8,1744889,1.0,Documentary|Comedy|Drama|Fantasy|Mystery|Sci-Fi,2004,What the #$*! Do We Know!?
1,8,1395430,2.0,Documentary|Comedy|Drama|Fantasy|Mystery|Sci-Fi,2004,What the #$*! Do We Know!?
2,8,1205593,4.0,Documentary|Comedy|Drama|Fantasy|Mystery|Sci-Fi,2004,What the #$*! Do We Know!?
3,8,1488844,4.0,Documentary|Comedy|Drama|Fantasy|Mystery|Sci-Fi,2004,What the #$*! Do We Know!?
4,8,1447354,1.0,Documentary|Comedy|Drama|Fantasy|Mystery|Sci-Fi,2004,What the #$*! Do We Know!?
...,...,...,...,...,...,...
2891053,17742,1898310,2.0,Biography|Drama|History|Romance,1995,Catherine the Great
2891054,17742,716096,4.0,Biography|Drama|History|Romance,1995,Catherine the Great
2891055,17742,38115,3.0,Biography|Drama|History|Romance,1995,Catherine the Great
2891056,17742,2646347,5.0,Biography|Drama|History|Romance,1995,Catherine the Great


In [12]:
import pandas as pd

def transform_file_to_dataframe(file_path):
    transformed_data = []
    with open(file_path, 'r') as file:
        current_movie_id = None  
        for line in file:
            line = line.strip()
            if line.endswith(':'):  
                # If current line is movie ID, strip colon to capture ID
                current_movie_id = line[:-1]  
            else:
                # Otherwise, split by commas and prepend most recent ID
                user_id, rating, date = line.split(',')
                transformed_data.append([current_movie_id, user_id, rating, date])

    columns = ['MovieID', 'UserID', 'Rating', 'Date']
    dataframe = pd.DataFrame(transformed_data, columns=columns)
    return dataframe

df_1 = transform_file_to_dataframe("/Users/kahaan/Desktop/multi-armed-bandits/netflix/combined_data_1.txt")
df_2 = transform_file_to_dataframe("/Users/kahaan/Desktop/multi-armed-bandits/netflix/combined_data_2.txt")
df_3 = transform_file_to_dataframe("/Users/kahaan/Desktop/multi-armed-bandits/netflix/combined_data_3.txt")
df_4 = transform_file_to_dataframe("/Users/kahaan/Desktop/multi-armed-bandits/netflix/combined_data_4.txt")

In [15]:
df_12 = pd.concat([df_1, df_2])
df_34 = pd.concat([df_3, df_4])
df_1234 = pd.concat([df_12, df_34])
df_1234

In [22]:
df_1234

,MovieID,UserID,Rating,Date
0,1,1488844,3,2005-09-06
1,1,822109,5,2005-05-13
2,1,885013,4,2005-10-19
3,1,30878,4,2005-12-26
4,1,823519,3,2004-05-03
...,...,...,...,...
26847518,17770,1790158,4,2005-11-01
26847519,17770,1608708,3,2005-07-19
26847520,17770,234275,1,2004-08-07
26847521,17770,255278,4,2004-05-28


In [23]:
entries_per_user = df_1234.groupby('UserID').size()
smallest_entries_count = entries_per_user.min()
print(f"The smallest amount of entries for any UserID is: {smallest_entries_count}")

The smallest amount of entries for any UserID is: 1


In [27]:
entries_per_user.median()

96.0